In [ ]:
NB_SIM = 1
QUANTILES = [0.95, 0.99, 0.997]

In [ ]:
import sys

sys.path.append('..')

In [ ]:
import os

In [ ]:
def rec_dict_print(a_dict, level=1):
    keys = a_dict.keys()
    keys.sort()
    for k in keys:
        if type(a_dict[k]) is dict:
            print "".join(["-" for i in range(level)]) + k
            rec_dict_print(a_dict[k], level + 2)
        else:
            print "".join(["-" for i in range(level)]) + k

In [ ]:
%matplotlib inline
import dill

loaded_data = None

with open('precomputed_sims/data%i.pkl' % (NB_SIM), 'rb') as f:
    loaded_data = dill.load(f)

In [ ]:
rec_dict_print(loaded_data)

In [ ]:
N = loaded_data["N"]

In [ ]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1 / 360.
delta = 5 * step

maturity = loaded_data["randomization"]["maturity"]

print "Maturity = %s years" % maturity

# Discount

In [ ]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

# Underlying

In [ ]:
udlyings = loaded_data["underlyings"]

print "Maximum number of paths: %i" % len(udlyings)

gbm0 = udlyings[0]

kappa = gbm0.drifts[0][0]
sigma = gbm0.vols[0][0]
print "kappa = %s, sigma = %s" % (kappa, sigma)

time_grid = gbm0.time

# Derivative

In [ ]:
derivatives_nb = 1

In [ ]:
from finance.products.european.swap import (
    SwapContract,
)

swap_delta = 0.25

swap_dates = SwapContract.generate_payment_dates(0, maturity, swap_delta)
swap = SwapContract(gbm0, discount, swap_dates)

price_0 = swap.price(0., incl_next_coupon=True)

print swap
print "\nPrice swap at t=0 without 1st coupon = %s" % price_0

In [ ]:
import numpy as np

p_fixed = 1.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

print "Notional on the swap: %s" % notional

# Indexes stuffs

In [ ]:
copula = loaded_data["credit"]["copula"]

c_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]

obligors_nb = len(copula.subsets[c_subsets_indexes[-1]][0])
print "Obligor numbers: %s" % obligors_nb

c_ids = [17, 9, 29, 26, 50, 4, 5, 13, 64]
c_positions = [0.69, -0.46, -0.44, -0.36, 0.34, 0.23, 0.09, -0.05, -0.04]

print "Counterparties id: %s (nb = %s)" % (c_ids, len(c_ids))

In [ ]:
positions = np.zeros(obligors_nb)
for idx, ps in zip(c_ids, c_positions):
    positions[idx] = ps

positions = positions / -positions[13]
positions = np.array(positions).reshape(positions.size, 1)

print positions.flatten()

## Default times

In [ ]:
default_times_mat = loaded_data["credit"]["default_times"]

In [ ]:
import pandas as pd

def get_default_times_sim(c_ids_, N_, default_times_mat_, copula_, subsets_indexes_):
    matrix_def_ = np.zeros((len(c_ids_), N_))
    
    for j_, c_id_ in enumerate(c_ids_):
        cp_subsets_indexes_ = copula_.get_indexes_including(c_id_)
        c_df_times_indexes_ = [ii_ for ii_, ind_ in enumerate(subsets_indexes_) if ind_ in cp_subsets_indexes_]
        
        c_df_times_mat_ = default_times_mat_[:, c_df_times_indexes_]
        c_df_times_ = c_df_times_mat_.min(axis=1)
        
        matrix_def_[j_] = c_df_times_
        
    matrix_def_[matrix_def_==1000.] = np.nan
        
    df_default_times_ = pd.DataFrame(matrix_def_, index=np.array(c_ids_))
    return df_default_times_

In [ ]:
default_times = get_default_times_sim(c_ids, N, default_times_mat, copula, c_subsets_indexes)
default_times

# Portfolio P&L

Code for:

\begin{equation*}
% \sum_{t < \tau_i^{\delta} \leq t+1} 
\left( 
\beta_{\tau_i^{\delta}} \left( MtM_{\tau_i^\delta}^i + \Delta^i_{\tau_i^\delta} \right)
-\beta_{\tau_i} \left( {\rm VM}_{\tau_i}^i + {\rm IM}_{\tau_i}^i \right)
\right)^+ \, \forall i
\end{equation*}

In [ ]:
from utils import time_offseter

In [ ]:
times_cva = np.arange(0, maturity, 0.5)

shifted_times_cva = times_cva + 1.
shifted_times_cva[-1] = maturity

default_times.index.values

In [ ]:
from functools import partial

def fact_(swap_, discount_, kappa_, delta_, t_):
    time_grid_ = swap_.underlying.time
    t_delta_ = time_offseter(t_ + delta_, time_grid_, True)
    
    coupon_dates_ = swap_.pillars
    l_t_delta_ = np.searchsorted(coupon_dates_, t_delta_, side='left')

    beta_T_l_ = map(discount_, coupon_dates_[l_t_delta_ + 1 :])    
    h_l_ = swap_.delta_time[l_t_delta_ : ]
    
    T_l_m1_ = kappa_ * coupon_dates_[l_t_delta_ : -1]
    exp_factor_ = map(np.exp, T_l_m1_)
    
    tmp_ = np.multiply(exp_factor_, h_l_)    
    res_ = np.dot(beta_T_l_, tmp_)
    
    return res_

fact = partial(fact_, swap, discount, kappa, delta)

In [ ]:
def compute_1y_ahead_loss_(fact_f_, kappa_, delta_, notional_, positions_, times_cva_, shifted_times_cva_, default_times_, udls_):
    ccp_loss_1y_ = dict()
    
    N_ = len(udls_)
    
    c_ids_ = default_times_.index.values
    
    for t0_, t1_ in zip(times_cva_, shifted_times_cva_):
        key_ = "[%.2f, %.2f]" % (t0_, t1_)
        print key_
        ccp_loss_1y_[key_] = pd.DataFrame(0., index=range(N_), columns=c_ids_)
        
        for j_ in xrange(N_):
            udl_ = udls_[j_]
            time_grid_ = udl_.time
            tau_j_ = default_times_.loc[c_ids_, j_]
            
            c_defaulted_ = tau_j_[(tau_j_ > t0_) & (tau_j_ <= t1_)]
            for (c_idx_, tau_) in c_defaulted_.iteritems():
                tau_ = time_offseter(tau_, time_grid_, True)
                tau_delta_ = time_offseter(tau_ + delta_, time_grid_, True)
                
                hat_s_tau_ = udl_(tau_)[0][0] * np.exp(kappa_ * tau_)
                hat_s_tau_delta_ = udl_(tau_delta_)[0][0] * np.exp(kappa_ * tau_delta_)
            
                loss_c_ = notional_ * fact_f_(tau_) * positions_[c_idx_][0] * (hat_s_tau_ - hat_s_tau_delta_)
                
                ccp_loss_1y_[key_].loc[j_, c_idx_] = loss_c_

    return ccp_loss_1y_
            
compute_1y_ahead_loss = partial(compute_1y_ahead_loss_, fact, kappa, delta, notional, positions)

In [ ]:
loss_path = './res/sim%i/loss_1y_ahead/' % (NB_SIM)

if not os.path.isdir(loss_path):
    os.makedirs(loss_path)
    one_y_ahead_loss = compute_1y_ahead_loss(times_cva, shifted_times_cva, default_times, udlyings)
    
    for k, dataframe_ in one_y_ahead_loss.iteritems():
        path = os.path.join(loss_path, 'loss_no_im%s.csv' % k)
        dataframe_.to_csv(path)

In [ ]:
from scipy.stats import norm

def B_(fact_f_, vol_, delta_, a_, omega_, t_):
    if a_ <= 0.5: 
        a_ = 1. - a_
    
    perc_ = a_ if omega_ <= 0. else (1. - a_)
    q_ = norm.ppf(perc_)
    
    var_ = vol_**2 * delta_
    exp_factor_ = np.exp(-0.5 * var_ + np.sqrt(var_) * q_)
    
    tmp_res_ = 1. - exp_factor_
    
    return np.sign(omega_) * tmp_res_ * fact_f_(t_)

B = partial(B_, fact, sigma, delta)

In [ ]:
def compute_disc_margins_(b_f_, kappa_, notional_, positions_, times_cva_, shifted_times_cva_, default_times_, udls_, q_):
    margins_ = dict()
    
    N_ = len(udls_)
    
    c_ids_ = default_times_.index.values
    
    print q_
    for t0_, t1_ in zip(times_cva_, shifted_times_cva_):
        key_ = "[%.2f, %.2f]" % (t0_, t1_)
        print key_
        margins_[key_] = pd.DataFrame(0., index=range(N_), columns=c_ids_)
        
        for j_ in xrange(N_):
            udl_ = udls_[j_]
            time_grid_ = udl_.time
            tau_j_ = default_times_.loc[c_ids_, j_]
            
            c_defaulted_ = tau_j_[(tau_j_ > t0_) & (tau_j_ <= t1_)]
            for (c_idx_, tau_) in c_defaulted_.iteritems():
                tau_ = time_offseter(tau_, time_grid_, True)
                                
                hat_s_tau_ = udl_(tau_)[0][0] * np.exp(kappa_ * tau_)
                
                nom_i = notional_ * np.abs(positions_[c_idx_][0])
                margin_ = nom_i * hat_s_tau_ * b_f_(q_, positions_[c_idx_][0], tau_)
            
                margins_[key_].loc[j_, c_idx_] = margin_

    return margins_

compute_disc_margins = partial(compute_disc_margins_, B, kappa, notional, positions)

In [ ]:
im_path = './res/sim%i/im/' % (NB_SIM)

for q in QUANTILES:
    im_q_path = os.path.join(im_path, str(q))
    if not os.path.isdir(im_q_path):
        os.makedirs(im_q_path)
        
    q_margins = compute_disc_margins(times_cva, shifted_times_cva, default_times, udlyings, q)
    for k, dataframe_ in q_margins.iteritems():
        path = os.path.join(im_q_path, 'im%s.csv' % k)
        dataframe_.to_csv(path)